# 載入所需套件

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re

# 定義抓取指定時間和恒指區域牛熊證街貨分佈資料

In [2]:
def bear_bull_crawler(date,code):
    url='https://www.bocifp.com/tc/cbbc/cbbc-hsi-os?date={}&spread={}'.format(date,code)
    response=requests.get(url)
    dfs=pd.read_html(response.text)
    
    soup=BeautifulSoup(response.text,'html.parser')
    #對沖張數(一日變化)提取()內一日變化數值
    bear_diff=[re.findall(r'.*\((.*)\).*',e.text)[0]  for index,e in enumerate(soup.select('.data')) if index<=13]
    bull_diff=[re.findall(r'.*\((.*)\).*',e.text)[0]  for index,e in enumerate(soup.select('.data')) if index>13]
    
    #中銀精選和收回價
    price_bear=dfs[3].iloc[:14,:]
    price_bull=dfs[3].iloc[15:,:]
    price_bull.reset_index(drop=True,inplace=True)
    
    #牛熊證比例
    bull_bear_perct=dfs[0]
    bull_bear_perct.columns=['標題','類別']
    bull_bear_perct['熊證比例']=bull_bear_perct['類別'].apply(lambda x:x.split('  ')[0])
    bull_bear_perct['牛證比例']=bull_bear_perct['類別'].apply(lambda x:x.split('  ')[1])
    bull_bear_perct=bull_bear_perct[['熊證比例','牛證比例']]
    
    df_bear=dfs[1].iloc[:15,:2]
    headers=df_bear.iloc[0]
    df_bear=pd.DataFrame(df_bear.values[1:],columns=headers)
    df_bear['收回價']=df_bear['熊證'].apply(lambda x:x.split('  ')[0].split(':')[1])
    df_bear['街貨量(百萬份)']=df_bear['熊證'].apply(lambda x:x.split('  ')[1].split(':')[1])
    df_bear['街貨相對期指張數(張)']=df_bear['熊證'].apply(lambda x:x.split('  ')[2].split(':')[1])
    df_bear.columns=['指數','熊證','收回價','街貨量(百萬份)','街貨相對期指張數(張)']
    df_bear['街貨相對期指張數一日變化(張)']=bear_diff
    df_bear=df_bear[['指數','收回價','街貨量(百萬份)','街貨相對期指張數(張)','街貨相對期指張數一日變化(張)']]
    
    
    df_bull=dfs[1].iloc[16:,:2]
    df_bull.columns=['指數','牛證']
    df_bull.reset_index(drop=True,inplace=True)
    df_bull.drop(df_bull.index[-1],inplace=True)
    df_bull['收回價']=df_bull['牛證'].apply(lambda x:x.split('  ')[0].split(':')[1])
    df_bull['街貨量(百萬份)']=df_bull['牛證'].apply(lambda x:x.split('  ')[1].split(':')[1])
    df_bull['街貨相對期指張數(張)']=df_bull['牛證'].apply(lambda x:x.split('  ')[2].split(':')[1])
    df_bull['街貨相對期指張數一日變化(張)']=bull_diff
    df_bull=df_bull[['指數','收回價','街貨量(百萬份)','街貨相對期指張數(張)','街貨相對期指張數一日變化(張)']]
    
    total_bear_df=pd.concat([df_bear,price_bear],axis=1)
    total_bull_df=pd.concat([df_bull,price_bull],axis=1)
    total_bear_bull_df=pd.concat([total_bear_df,total_bull_df],axis=0,ignore_index=True)
    
    return bull_bear_perct,total_bear_df,total_bull_df,total_bear_bull_df

# 指定恒指區域抓取當天牛熊證街貨分佈資料

In [3]:
date=datetime.now().strftime('%Y-%m-%d')

diff_dict={'50':'0',
          '100':'1',
          '200':'2',
          '300':'3',
          '400':'4',
          '500':'5'}

condition=True
while condition:
    try:
        diff=str(input('輸入恒指區域 (50, 100, 200, 300, 400, 500) : '))
        code=diff_dict[diff]
        condition=False
    except:
        print('請輸入正常範圍 !!')

bull_bear_perct,total_bear_df,total_bull_df,total_bear_bull_df=bear_bull_crawler(date,code)

輸入恒指區域 (50, 100, 200, 300, 400, 500) : 150
請輸入正常範圍 !!
輸入恒指區域 (50, 100, 200, 300, 400, 500) : 200


# 牛熊證比例表

In [4]:
bull_bear_perct

,熊證比例,牛證比例
0,42.6%,57.4%


# 熊證街貨分佈資料

In [5]:
total_bear_df

,指數,收回價,街貨量(百萬份),街貨相對期指張數(張),街貨相對期指張數一日變化(張),中銀精選,收回價
0,"28,300 - 28,499","28,300至28,499",20.79,21.45,4,NaN,NaN
1,"28,100 - 28,299","28,100至28,299",4.70,9.39,-0,NaN,NaN
2,"27,900 - 28,099","27,900至28,099",9.99,10.82,3,NaN,NaN
3,"27,700 - 27,899","27,700至27,899",15.92,21.36,1,NaN,NaN
4,"27,500 - 27,699","27,500至27,699",70.31,75.99,11,54215.0,27600.0
5,"27,300 - 27,499","27,300至27,499",90.36,176.90,44,53201.0,27300.0
6,"27,100 - 27,299","27,100至27,299",111.08,199.04,60,53425.0,27100.0
7,"26,900 - 27,099","26,900至27,099",247.92,264.41,-49,51600.0,27000.0
8,"26,700 - 26,899","26,700至26,899",453.92,820.75,86,53898.0,26858.0
9,"26,500 - 26,699","26,500至26,699",279.68,527.36,56,55314.0,26500.0


# 牛證街貨分佈資料

In [6]:
total_bull_df

,指數,收回價,街貨量(百萬份),街貨相對期指張數(張),街貨相對期指張數一日變化(張),中銀精選,收回價
0,"25,100 - 25,299","25,100至25,299",221.65,424.19,-70,63134.0,25100.0
1,"24,900 - 25,099","24,900至25,099",241.46,433.06,-438,67084.0,24900.0
2,"24,700 - 24,899","24,700至24,899",159.32,306.33,-16,67086.0,24700.0
3,"24,500 - 24,699","24,500至24,699",441.59,843.95,-592,63139.0,24500.0
4,"24,300 - 24,499","24,300至24,499",155.98,290.37,-265,67221.0,24300.0
5,"24,100 - 24,299","24,100至24,299",405.64,797.83,-159,61701.0,24250.0
6,"23,900 - 24,099","23,900至24,099",744.15,"1,313.98",299,64365.0,23950.0
7,"23,700 - 23,899","23,700至23,899",609.99,"1,177.41",-37,61516.0,23700.0
8,"23,500 - 23,699","23,500至23,699",821.38,"1,498.44",-100,59276.0,23500.0
9,"23,300 - 23,499","23,300至23,499",123.81,216.05,-35,63222.0,23450.0


# 牛熊證街貨分佈資料

In [7]:
total_bear_bull_df

,指數,收回價,街貨量(百萬份),街貨相對期指張數(張),街貨相對期指張數一日變化(張),中銀精選,收回價
0,"28,300 - 28,499","28,300至28,499",20.79,21.45,4,NaN,NaN
1,"28,100 - 28,299","28,100至28,299",4.70,9.39,-0,NaN,NaN
2,"27,900 - 28,099","27,900至28,099",9.99,10.82,3,NaN,NaN
3,"27,700 - 27,899","27,700至27,899",15.92,21.36,1,NaN,NaN
4,"27,500 - 27,699","27,500至27,699",70.31,75.99,11,54215.0,27600.0
5,"27,300 - 27,499","27,300至27,499",90.36,176.90,44,53201.0,27300.0
6,"27,100 - 27,299","27,100至27,299",111.08,199.04,60,53425.0,27100.0
7,"26,900 - 27,099","26,900至27,099",247.92,264.41,-49,51600.0,27000.0
8,"26,700 - 26,899","26,700至26,899",453.92,820.75,86,53898.0,26858.0
9,"26,500 - 26,699","26,500至26,699",279.68,527.36,56,55314.0,26500.0


# 將指定恒指區域當天牛熊證街貨分佈資料匯出成Excel檔

In [8]:
total_bear_bull_df.to_excel('{}恒指區域{}牛熊證街貨分佈圖.xlsx'.format(date,diff),index=False)